### Задание
Проанализировать массив данных при помощи языка Python (допускается и рекомендуется использование дополнительных библиотек).
1. Для числовых значений как для всего массива в целом, так и для каждого типа контента (столбец Type) в отдельности вычислить:
 * вычисление среднего
 * максимального/минимального значений
 * медианы
 * моды  
2. Найти самый популярный объект в выборке, объяснить почему.


In [1]:
import pandas as pd
from scipy import stats
import numpy as np
from sklearn.preprocessing import LabelEncoder
from statistics import median
from sklearn.preprocessing import MinMaxScaler

Загрузим необходимый dataset, определим его размер, просмотрим атрибуты и проанализируем данные

### Подготовка данных

In [2]:
df = pd.read_csv('dataset_Facebook.csv', sep=';')

In [3]:
print('Размер:', df.shape)
print('Атрибуты: \n', df.columns)

Размер: (500, 19)
Атрибуты: 
 Index(['Page total likes', 'Type', 'Category', 'Post Month', 'Post Weekday',
       'Post Hour', 'Paid', 'Lifetime Post Total Reach',
       'Lifetime Post Total Impressions', 'Lifetime Engaged Users',
       'Lifetime Post Consumers', 'Lifetime Post Consumptions',
       'Lifetime Post Impressions by people who have liked your Page',
       'Lifetime Post reach by people who like your Page',
       'Lifetime People who have liked your Page and engaged with your post',
       'comment', 'like', 'share', 'Total Interactions'],
      dtype='object')


Проверим Dataframe на наличие неопределенных значений и в случае их наличия проинициализируем их нулями

In [4]:
print(df.isnull().sum())
if df.isnull().values.any:
    df.fillna(0, inplace=True)
    print(' ********* Данные после обработки **********')
    print(df.isnull().sum())

Page total likes                                                       0
Type                                                                   0
Category                                                               0
Post Month                                                             0
Post Weekday                                                           0
Post Hour                                                              0
Paid                                                                   1
Lifetime Post Total Reach                                              0
Lifetime Post Total Impressions                                        0
Lifetime Engaged Users                                                 0
Lifetime Post Consumers                                                0
Lifetime Post Consumptions                                             0
Lifetime Post Impressions by people who have liked your Page           0
Lifetime Post reach by people who like your Page   

### Произведем вычисление необходимых параметров

Так как необходимо вычислить парметры всех признаков, в том числе столбца Type, который является категориальным, нам необходимо сопоставить каждый тип с числом, сделаем это с помощью LabelEncoder.  
Следует учесть, что для обучения на данных лучше использовать другие способы кодирования, например dummy-кодирование, так как этот способ сильно упрощает данные, просто проецируя категориальный признак на вещественную прямую  
Зададим словарь для кодировки вручную, сразу же посмотрим, какие типы у нас имеются и в будущем сможем легко перевести закодированный параметр обратно

In [5]:
categories_dict = {}
categories = list(np.unique(df.Type))
for index, value in enumerate(categories):
    categories_dict.update({value: index + 1})
print('Закодированные категориальные признаки', categories_dict)


Закодированные категориальные признаки {'Link': 1, 'Photo': 2, 'Status': 3, 'Video': 4}


In [6]:
# Функция для получения типа поста по значению
def get_Type(value):
    value = int(value)
    for category, v in categories_dict.items():
        if v == value:
            return category

In [7]:
label_encoder = LabelEncoder()
label_encoder.fit(df.Type)
df['Type_encoded'] = df['Type'].map(categories_dict)
df.drop(['Type'], axis=1, inplace=True)

Разработаем функцию для вычисления всех необходимых параметров каждого столбца, которая будет принимать на вход текущий столбец и возвращать его характеристики

In [8]:
def analize_column(column):
    results = {}
    results.update({'mean': np.mean(column)}) 
    results.update({'min': np.min(column)}) 
    results.update({'max': np.max(column)}) 
    results.update({'median': median(column)}) 
    results.update({'mode' : list(stats.mode(column).mode)[0]})
    return results

Создадим новый Dataframe, который будем использовать для наглядной демонстрации полученных результатов

In [9]:
result_df = pd.DataFrame(columns=df.columns, index=['mean', 'min', 'max', 'mode', 'median'])

Проведем подсчет характеристик

In [10]:
for attribute in df.columns:
    result = analize_column(df[attribute])
    for parameter in result_df.index:
        result_df.loc[parameter][attribute] = result.get(parameter)

Получаем Dataframe со всеми необходимыми параметрами

In [11]:
result_df

,Page total likes,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions,Type_encoded
mean,123194,1.88,7.038,4.15,7.84,0.278,13903.4,29585.9,920.344,798.772,1415.13,16766.4,6585.49,609.986,7.482,177.59,27.048,212.12,2.074
min,81370,1,1,1,1,0,238,570,9,9,9,567,236,9,0,0,0,0,1
max,139441,3,12,7,23,1,180480,1110282,11452,11328,19779,1107833,51456,4376,372,5172,790,6334,4
mode,136393,1,10,7,3,0,677,4372,537,182,431,1210,690,403,0,98,13,0,2
median,129600,2,7,4,9,0,5281,9051,625.5,551.5,851,6255.5,3417,412,3,101,19,123.5,2


Теперь вычислим все необходимые параметры для всего массива данных

In [12]:
values_list = []
for attribute in df.columns:
    values_list += list(df[attribute].values)
if df.shape[0] * df.shape[1] == len(values_list):
    print("Все данные собраны в список")
else: 
    print("Количество данных в списке и в Dataframe не совпадают")

Все данные собраны в список


In [13]:
print('Характеристики по всему Dataframe')
result_df_parameters = {
    'mean': format("%.2f" % np.mean(values_list)),
    'min': format("%.2f" % np.min(values_list)),
    'max': format("%.2f" % np.max(values_list)),
    'median': format("%.2f" % median(values_list)),
    'mode': format("%.2f" % list(stats.mode(values_list).mode)[0])
}
result_df_parameters

Характеристики по всему Dataframe


{'max': '1110282.00',
 'mean': '10222.48',
 'median': '155.50',
 'min': '0.00',
 'mode': '2.00'}

### Найдем самый популярный объект в выборке

In [14]:
df.head()

,Page total likes,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions,Type_encoded
0,139441,2,12,4,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0,100,2
1,139441,2,12,3,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0,164,3
2,139441,3,12,3,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0,80,2
3,139441,2,12,2,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0,1777,2
4,139441,2,12,2,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0,393,2


Рассмотрев имеющиеся данные следует выделить ключевые признаки, по которым можно оценить популярность объекта.  
Я считаю, что такими ключевыми признаками популярности являются:  
* Репосты  
* Комментарии
* Лайки  

Так же стоит учитывать то, что эти признаки вносят неравный вклад, очевидно, что наибольший вклад будет вность количество репостов, затем, с небольшим перевесом, лайки и, наконец, комментарии (т.к. множество комментариев может оставить один аккаунт).  
Итак, нормализуем данные по трем ключевым параметрам и разработаем функцию, вычисляющую популярность объекта, предположив, что признаки имеют следующие веса:  
* Репосты - 0.6
* Лайки - 0.3
* Комментарии - 0.1

In [15]:
mms = MinMaxScaler()
df['normalized_likes'] = mms.fit_transform(df.like.values.reshape(-1, 1))
df['normalized_comments'] = mms.fit_transform(df.comment.values.reshape(-1, 1))
df['normalized_shares'] = mms.fit_transform(df.share.values.reshape(-1, 1))

C:\Users\Gleb\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [16]:
def evaluate(sample):
    return sample.normalized_likes * 0.3 + sample.normalized_comments * 0.1 + sample.normalized_shares * 0.6

In [17]:
result = 0
post = 0
for i in range(df.shape[0]):
    mark = evaluate(df.loc[i])
    if mark > result:
        result = mark
        post = i
        
print('наиболее популярный пост: \n'
     'Номер поста - ', post, '\n'
     'Тип контента - ', get_Type(df.loc[post].Type_encoded), '\n')
print(df.loc[post])

наиболее популярный пост: 
Номер поста -  244 
Тип контента -  Photo 

Page total likes                                                       130791.0
Category                                                                    2.0
Post Month                                                                  7.0
Post Weekday                                                                3.0
Post Hour                                                                   5.0
Paid                                                                        1.0
Lifetime Post Total Reach                                              180480.0
Lifetime Post Total Impressions                                        319133.0
Lifetime Engaged Users                                                   8072.0
Lifetime Post Consumers                                                  4010.0
Lifetime Post Consumptions                                               6242.0
Lifetime Post Impressions by people who have like

### Итоги работы
В ходе проведенной работы были найдены среднее, минимальное, максимальное, медиана и мода для каждого признака и для массива данных в целом. Для вычисления этих параметров для признаков была написана функция, возвращающая словарь с искомыми значениями, однако стоит учитывать то, что такие параметры, как среднее, минимум, максимум для каждого признака Dataframe можно вычислить намного проще, используя соответствующие функции библиотеки numpy и применяя их к Dataframe.  
Также была предложена формула для оценки популярности поста, имеющая 3 слагаемых (лайки, комментарии и репосты),значения которых были нормализованы и взяты с определенными весами.  
Для наиболее популярного поста были показаны его номер, тип контента и значения всех его признаков.